In [53]:
pip install yfinance pandas ta openpyxl

Note: you may need to restart the kernel to use updated packages.


In [55]:
import yfinance as yf
import pandas as pd
import ta
from openpyxl import load_workbook
from openpyxl.styles import Font

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

companies = {
    "AAPL": "Apple",
    "HSBC": "HSBC",
    "PEP": "Pepsi",
    "TM": "Toyota",
    "TCEHY": "Tencent"
}

start_date = "2016-05-01"
end_date = "2024-05-08"

frames = []

for ticker, name in companies.items():
    df = yf.download(ticker, start=start_date, end=end_date)

    if isinstance(df.columns, pd.MultiIndex):
        new_columns = []
        for col in df.columns:
            if isinstance(col, tuple):
                new_columns.append("_".join([str(c) for c in col]).strip())
            else:
                new_columns.append(col)
        df.columns = new_columns

    df = df.reset_index()
    df["Company"] = name
    df = df.sort_values("Date").copy()

    close_col = next((col for col in df.columns if "Close" in col), "Close")
    volume_col = next((col for col in df.columns if "Volume" in col), "Volume")

    close = df[close_col].squeeze()
    close.index = df.index
    volume = df[volume_col].squeeze()
    volume.index = df.index

    df["SMA_5"] = ta.trend.SMAIndicator(close=close, window=5).sma_indicator()
    df["SMA_20"] = ta.trend.SMAIndicator(close=close, window=20).sma_indicator()
    df["EMA_10"] = ta.trend.EMAIndicator(close=close, window=10).ema_indicator()

    df["Momentum_3"] = ta.momentum.ROCIndicator(close=close, window=3).roc()
    df["Momentum_10"] = ta.momentum.ROCIndicator(close=close, window=10).roc()

    df["RSI_14"] = ta.momentum.RSIIndicator(close=close, window=14).rsi()

    macd = ta.trend.MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD"] = macd.macd()
    df["MACD_Signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close=close, window=20, window_dev=2)
    df["BB_Upper"] = bb.bollinger_hband()
    df["BB_Lower"] = bb.bollinger_lband()

    df["OBV"] = ta.volume.OnBalanceVolumeIndicator(close=close, volume=volume).on_balance_volume()

    frames.append(df)

combined_df = pd.concat(frames, ignore_index=True)
combined_df = combined_df.sort_values(["Company", "Date"])

indicator_cols = ["SMA_5","SMA_20","EMA_10","Momentum_3","Momentum_10","RSI_14",
                  "MACD","MACD_Signal","BB_Upper","BB_Lower","OBV"]
original_cols = [col for col in combined_df.columns if col not in indicator_cols]

combined_df = combined_df[original_cols + indicator_cols]

output_file = "combined_technical_indicators.xlsx"
combined_df.to_excel(output_file, index=False, engine='openpyxl')

wb = load_workbook(output_file)
ws = wb.active

for cell in ws[1]:
    cell.font = Font(bold=True)

for column_cells in ws.columns:
    max_length = 0
    column_letter = column_cells[0].column_letter
    for cell in column_cells:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column_letter].width = max_length + 2

wb.save(output_file)
print(f"Data saved and formatted in {output_file}")

/tmp/ipykernel_3549/3120697970.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_3549/3120697970.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_3549/3120697970.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_3549/3120697970.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_3549/3120

Data saved and formatted in combined_technical_indicators.xlsx
